In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.spatial.distance import cdist
from IPython.display import display, Math

In [ ]:
def generatePopulation(n, p, q):
    return np.concatenate((
        p[0] + np.random.rand(1,n) * abs(p[0]-p[1]),
        q[0] + np.random.rand(1,n) * abs(q[0]-q[1])))

generatePopulation(5, [0, 1], [-100, 0])

In [ ]:
def triangle_center(tri):
    return np.array([(tri[0][0] + tri[1][0] + tri[2][0]) / 3,
                     (tri[0][1] + tri[1][1] + tri[2][1]) / 3])

In [ ]:
def plot_triangle(triangle):
    plt.plot(*np.array(triangle).T, 'o', color='blue')
    plt.plot(*triangle_center(triangle), '*', color='red')

In [ ]:
triangle = [[1,1], [0,0], [2,-1]]

plot_triangle(triangle)
plt.show()

In [ ]:
def dist2D(p):
    dist = 0
    for k in p:
        for l in p:
            new_dist = np.linalg.norm(k - l)
            if new_dist > dist:
                dist = new_dist
    
    return dist

In [ ]:
def plotContour(f, x1_int, x2_int, argmin=None, mesh_n=50):
    f_ = lambda x1, x2: f([x1, x2])
    
    x = np.linspace(x1_int[0], x1_int[1], mesh_n)
    y = np.linspace(x2_int[0], x2_int[1], mesh_n)

    X, Y = np.meshgrid(x, y)
    Z = f_(X, Y)

    fig = plt.figure(figsize=(7, 4))
    ax = fig.add_subplot()

    cs = ax.contourf(X, Y, Z)

    fig.colorbar(cs)

    if(argmin != None):
        ax.plot([argmin[0]], [argmin[1]], 'r*', label=r'$f(x) = min $')

    plt.show()

# Ackley
f = lambda x: -20*np.exp(-0.2*np.sqrt(0.5*(x[0]*x[0]+x[1]*x[1]))) \
                   -np.exp(0.5*(np.cos(2*np.pi*x[0])+np.cos(2*np.pi*x[1]))) \
                   +20+np.e

p = q = [-5, 5]

# Bukin6
f = lambda x: 100*np.sqrt(np.abs(x[1]-0.01*np.power(x[0],2)))+0.01*np.abs(x[0]+10)

p = [-15, 5]
q = [-3, 3]

In [ ]:
# Holder
f = lambda x: -np.abs(np.sin(x[0])*np.cos(x[1])*np.exp(np.abs(1-(np.sqrt(x[0]*x[0]+x[1]*x[1])/np.pi))))

p = q = [-10, 10]

In [ ]:
plotContour(f, p, q)

In [ ]:
n = 600
max_iter = 100
eps = 1e-6
delta = 1e-9
a = 3
alpha = np.pi/4

In [ ]:
Pt = generatePopulation(n, p, q)
Pt

In [ ]:
Ft = f(Pt)
Ft

In [ ]:
t = 0
Pt = generatePopulation(n, p, q)
while True:
    #S3

    # ------------------------------ Pz -----------------------------------
    np.random.shuffle(Pt)
    Ft = f(Pt)
    
    #print(Pt.shape)
    #print(dist2D(Pt))
    #print(dist2D(Ft))
    #print(t)
    
    if t == max_iter or dist2D(Pt) < eps or dist2D(Ft) < delta:
        print("t = ", t)
        print(np.min(Ft), "in the point", Pt[:,np.argmin(Ft)])
        x_min = Pt[:,np.argmin(Ft)]
        break
    
    to_triangles = lambda p: p.reshape(int(n/3), 3, 2).copy()
    to_points = lambda triangles: triangles.reshape(n,2).T.copy()
    
    
    triangles = to_triangles(Pt)
    
    for tri in triangles:
        #print(t)
        tFt = f(tri.T) #S2
        #print(tFt)
        center = triangle_center(tri) #S4
        
        best_index = np.argmax(tFt) #S5
        #print(t[best_index])

        for i in range(3):
            if i == best_index:
                tri[i] = (1 + a)*tri[i] - a*center #S6
            else:
                tri[i] = (tri[best_index] + a*tri[i])/(1+a) #S7
        
        #print(t)
        
        #plot_triangle(t)
    #plt.show()
    #print(triangles)
    Pz = to_points(triangles)
    #print(Pz)
    np.random.shuffle(Pz)
    # ------------------------------ Pw -----------------------------------
    #S8
    triangles = to_triangles(Pz)
    
    for tri in triangles:
        center = triangle_center(tri)
        for i in range(3):
            tri[i][0] = (tri[i][0] - center[0])*np.cos(alpha) - (tri[i][1] - center[1])*np.sin(alpha) + center[0]
            tri[i][1] = (tri[i][0] - center[0])*np.sin(alpha) + (tri[i][1] - center[1])*np.cos(alpha) + center[1]
            
    Pw = to_points(triangles)
    
    #assert(False)
    
    # ----------------------------- P --------------------------------------
    P = np.concatenate((Pt, Pz, Pw), axis=1)
    
    #print(p, q)
    #print(np.clip(P[0], p[0], p[1]))
    P[0] = np.clip(P[0], q[0], q[1])
    #print(P[0])
    P[1] = np.clip(P[1], p[0], p[1])
    # keep points in the domain
    P
    
    #print(P.shape)
    F = f(P)
    #print(F.shape)
    
    sorted_index = np.argsort(F)
    Pt = P[:,sorted_index[:n]] #min
    #Pt = P[:,sorted_index[-n:]] #max
    #print(Pt.shape)
    t += 1
    if t % 20 == 0: print(t)


In [ ]:
plotContour(f, p, q, x_min.tolist())